In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score, precision_score
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
import re


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, GlobalMaxPooling1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding


In [2]:
#Validation set manually annotated 
Validationdata = pd.read_csv('data/ds.tsv',delimiter="\t")

#Set of posts samples (positive/control) filtered by their topical similarity with a depression taxonomy (Ht)
HT_data = pd.read_csv('data/ts_ht.tsv',delimiter="\t")


In [3]:

X_train = HT_data['pp_text']
X_test = Validationdata['pp_text']
y_train = HT_data['label']
y_test = Validationdata['label']
y_test

0      1
1      1
2      1
3      1
4      1
      ..
143    0
144    0
145    0
146    0
147    0
Name: label, Length: 148, dtype: int64

In [4]:
# defining function to clean text and retrive closs-validation datasets
def cleantxt(txt):
    """
    Cleans the string passed. Cleaning Includes-
    1. remove special characters/symbols
    2. convert text to lower-case
    3. retain only alphabets
    4. remove words less than 3 characters
    5. remove stop-words
    """  
    # collecting english stop words from nltk-library
    stpw = stopwords.words('english')
    
    # Adding custom stop-words
    stpw.extend(['www','http','utc'])
    stpw = set(stpw)
    
    # using regex to clean the text
    txt = re.sub("[\<\[].*?[\>\]]", " ", txt)
    txt = txt.lower()
    txt = re.sub(r"[^a-z ]", " ", txt)
    txt = " ".join([x for x in txt.split() if x not in stpw])
    return txt




In [5]:

for text in X_train:
    cleantxt(text)
    
for text in X_test:
    cleantxt(text)

In [6]:
   
tokenizer = Tokenizer(num_words=6948)

tokenizer.fit_on_texts(X_train)
list_tokenized_train = tokenizer.texts_to_sequences(X_train)

tokenizer.fit_on_texts(X_test)
list_tokenized_test = tokenizer.texts_to_sequences(X_test)



X_t = pad_sequences(list_tokenized_train, 165)
X_test = pad_sequences(list_tokenized_test, 165)



X_val = X_t[:3000]
X_trainn = X_t[3000:]

y_val = y_train[:3000]
y_trainn = y_train[3000:]

y_trainn = np.asarray(y_trainn)
y_val = np.asarray(y_val)

print('Shape of X train and X validation tensor:', X_trainn.shape,X_val.shape)
print('Shape of label train and validation tensor:', y_trainn.shape,y_val.shape)

Shape of X train and X validation tensor: (4000, 165) (3000, 165)
Shape of label train and validation tensor: (4000,) (3000,)


In [7]:
max_features = 100000
batch_size = 128
epochs = 5
num_classes=5
embed_size = 150
num_unique_word = 6948

In [16]:
## create model
model_LSTM = Sequential()
model_LSTM.add(Embedding(num_unique_word, embed_size))
model_LSTM.add(Conv1D(64,kernel_size=4,padding='same',activation='relu',strides=1))
model_LSTM.add(MaxPooling1D(pool_size=4))
model_LSTM.add(LSTM(100))
model_LSTM.add(Dense(1, activation='relu'))
model_LSTM.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit train data
model_LSTM.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 150)         1042200   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 64)          38464     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, None, 64)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 1,146,765
Trainable params: 1,146,765
Non-trainable params: 0
_________________________________________________________________


In [17]:
hist = model_LSTM.fit(X_trainn,y_trainn, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val))

Epoch 1/5
32/32 [==============================] - 4s 72ms/step - loss: 1.2508 - accuracy: 0.5534 - val_loss: 0.5457 - val_accuracy: 0.7113
Epoch 2/5
32/32 [==============================] - 2s 60ms/step - loss: 0.4718 - accuracy: 0.7662 - val_loss: 0.4901 - val_accuracy: 0.7677
Epoch 3/5
32/32 [==============================] - 2s 60ms/step - loss: 0.3534 - accuracy: 0.8522 - val_loss: 0.4978 - val_accuracy: 0.8230
Epoch 4/5
32/32 [==============================] - 2s 61ms/step - loss: 0.2550 - accuracy: 0.9147 - val_loss: 0.4620 - val_accuracy: 0.8473
Epoch 5/5
32/32 [==============================] - 2s 61ms/step - loss: 0.1477 - accuracy: 0.9609 - val_loss: 0.6836 - val_accuracy: 0.8593


In [18]:
results = model_LSTM.evaluate(X_test,Validationdata['label'])

5/5 [==============================] - 0s 5ms/step - loss: 2.6218 - accuracy: 0.6757


In [27]:
model2 = Sequential()
model2.add(Embedding(num_unique_word, embed_size))
model2.add(Conv1D(128,kernel_size=5,padding='same',activation='relu',strides=1))
model2.add(MaxPooling1D(pool_size=4))
model2.add(LSTM(100, return_sequences = True))
model2.add(Dense(1, activation="relu"))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [28]:
hist = model2.fit(X_trainn,y_trainn, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val))

Epoch 1/5
32/32 [==============================] - 5s 94ms/step - loss: 1.1276 - accuracy: 0.4901 - val_loss: 0.6521 - val_accuracy: 0.6380
Epoch 2/5
32/32 [==============================] - 3s 79ms/step - loss: 0.6355 - accuracy: 0.6415 - val_loss: 0.6303 - val_accuracy: 0.6452
Epoch 3/5
32/32 [==============================] - 3s 79ms/step - loss: 0.6012 - accuracy: 0.6536 - val_loss: 0.6189 - val_accuracy: 0.6519
Epoch 4/5
32/32 [==============================] - 3s 80ms/step - loss: 0.5934 - accuracy: 0.6524 - val_loss: 0.6509 - val_accuracy: 0.6306
Epoch 5/5
32/32 [==============================] - 3s 81ms/step - loss: 0.5739 - accuracy: 0.6654 - val_loss: 0.6616 - val_accuracy: 0.6349


In [29]:
results = model2.evaluate(X_test,Validationdata['label'])

5/5 [==============================] - 0s 7ms/step - loss: 0.9957 - accuracy: 0.6501


In [23]:

predictedsvm = model_LSTM.predict_classes(X_test)
predictedsvm = predictedsvm[:, 0]
accuracy_score_sig = metrics.accuracy_score(predictedsvm, y_test)
accuracy_score_auc = metrics.roc_auc_score(predictedsvm, y_test)
accuracy_score_priv = metrics.precision_score(predictedsvm, y_test)
accuracy_score_recall = metrics.recall_score(predictedsvm, y_test)

print(accuracy_score_priv)
print(accuracy_score_recall)
print(accuracy_score_auc)
accuracy_score_sig

C:\Users\calum\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.8363636363636363
0.5411764705882353
0.6991596638655462


0.6756756756756757